#### Import required packages

In [134]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
from torch.autograd import Variable

#### Import movies dataset
<u>Column Meaning:</u><br>
<i>0 - Movie ID<br>
1 - Name<br>
2 - Genres</i>

In [135]:
movies = pd.read_csv('ml-1m/movies.dat', sep='::', header = None, engine = 'python', encoding = 'latin-1')
movies.head()

,0,1,2
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


#### Import users dataset
<u>Column Meaning:</u><br>
<i>0 - User ID<br>
1 - Gender<br>
2 - Age<br>
3 - Job<br>
4 - Zip Code</i>

In [136]:
users = pd.read_csv('ml-1m/users.dat', sep='::', header = None, engine = 'python', encoding = 'latin-1')
users.head()

,0,1,2,3,4
0,1,F,1,10,48067
1,2,M,56,16,70072
2,3,M,25,15,55117
3,4,M,45,7,02460
4,5,M,25,20,55455


#### Import ratings dataset
<u>Column Meaning:</u><br>
<i>0 - User ID<br>
1 - Movie ID<br>
2 - Ratings (1-5)<br>
3 - Timestamp when movie was rated</i>

In [137]:
ratings = pd.read_csv('ml-1m/ratings.dat', sep='::', header = None, engine = 'python', encoding = 'latin-1')
ratings.head()

,0,1,2,3
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


#### Import training set of 80,000 movie ratings and convert to Numpy array
<u>Column Meaning:</u><br>
<i>0 - User ID<br>
1 - Movie ID<br>
2 - Ratings (1-5)<br>
3 - Timestamp when movie was rated</i>

In [138]:
training_set = pd.read_csv('ml-100k/u1.base', header = None, sep = '\t')
training_set = np.array(training_set, dtype=int)
training_set

array([[        1,         1,         5, 874965758],
       [        1,         2,         3, 876893171],
       [        1,         3,         4, 878542960],
       ...,
       [      943,      1188,         3, 888640250],
       [      943,      1228,         3, 888640275],
       [      943,      1330,         3, 888692465]])

#### Import test set of 20,000 movie ratings and convert to Numpy array
<u>Column Meaning:</u><br>
<i>0 - User ID<br>
1 - Movie ID<br>
2 - Ratings (1-5)<br>
3 - Timestamp when movie was rated</i>

In [139]:
test_set = pd.read_csv('ml-100k/u1.test', header = None, sep = '\t')
test_set = np.array(test_set, dtype=int)
test_set

array([[        1,         6,         5, 887431973],
       [        1,        10,         3, 875693118],
       [        1,        12,         5, 878542960],
       ...,
       [      459,       934,         3, 879563639],
       [      460,        10,         3, 882912371],
       [      462,       682,         5, 886365231]])

#### Get the max user ID

In [140]:
num_users = int(max(max(training_set[:, 0]), max(test_set[:,0])))
num_users

943

#### Get the max movie ID

In [141]:
num_movies = int(max(max(training_set[:, 1]), max(test_set[:,1])))
num_movies

1682

#### Create a function that will convert the data into an array with users in lines and movies in columns

In [142]:
def convert(data):
    new_data = []
    for user_id in range(1, num_users + 1):
        movie_ids = data[:,1][data[:, 0] == user_id]
        rating_ids = data[:,2][data[:, 0] == user_id]
        ratings = np.zeros(num_movies)
        # Movie IDs start at 1 so we need to subtract 1 to get to base 0 index
        ratings[movie_ids - 1] = rating_ids
        new_data.append(list(ratings))
    return new_data

In [143]:
training_set = convert(training_set)

In [144]:
test_set = convert(test_set)

#### Converting data into Torch tensor

In [145]:
training_set = torch.FloatTensor(training_set)

In [146]:
test_set = torch.FloatTensor(test_set)

#### Converting the movie ratings (from 1 - 5) into binary ratings (1 if 'liked' and 0 if 'not liked')

In [147]:
training_set[training_set == 0] = -1
training_set[training_set == 1] = 0
training_set[training_set == 2] = 0
training_set[training_set >= 3] = 1
test_set[test_set == 0] = -1
test_set[test_set == 1] = 0
test_set[test_set == 2] = 0
test_set[test_set >= 3] = 1

#### Creating the Restricted Boltzmann Machine Neural Network Architecture

In [148]:
class RBM():
    def __init__(self, num_visible, num_hidden):
        self.Weights = torch.randn(num_hidden, num_visible)
        self.bias_a = torch.randn(1, num_hidden)
        self.bias_b = torch.randn(1, num_visible)
    def sample_hidden(self, x):
        wx = torch.mm(x, self.Weights.t())
        activation = wx + self.bias_a.expand_as(wx)
        prob_hidden_given_visible = torch.sigmoid(activation)
        return prob_hidden_given_visible, torch.bernoulli(prob_hidden_given_visible)
    def sample_visible(self, y):
        wy = torch.mm(y, self.Weights)
        activation = wy + self.bias_b.expand_as(wy)
        prob_visible_given_hidden = torch.sigmoid(activation)
        return prob_visible_given_hidden, torch.bernoulli(prob_visible_given_hidden)
    def train(self, v0, vk, ph0, phk):
        self.Weights += (torch.mm(v0.t(), ph0) - torch.mm(vk.t(), phk)).t()
        self.bias_b += torch.sum((v0-vk), 0)
        self.bias_a += torch.sum((ph0-phk), 0)

#### Set prelim input parameters

In [149]:
num_visible = len(training_set[0])
num_hidden = 100 # Can be tuned
batch_size = 100 # Can be tuned

#### Create instance of RBM model

In [150]:
rbm = RBM(num_visible, num_hidden)

#### Training the RBM

In [151]:
num_epochs = 10

In [152]:
for epoch in range(1, num_epochs + 1):
    train_loss = 0
    s = 0. # Counter
    for user_id in range(0, num_users - batch_size, batch_size):
        vk = training_set[user_id:user_id + batch_size]
        v0 = training_set[user_id:user_id + batch_size]
        ph0, _ = rbm.sample_hidden(v0)
        for k in range(10):
            _, hk = rbm.sample_hidden(vk)
            _, vk = rbm.sample_visible(hk)
            vk[v0<0] = v0[v0<0] # Freeze sells containing -1 ratings so they do not influence learning
        phk, _ = rbm.sample_hidden(vk)
        rbm.train(v0, vk, ph0, phk)
        train_loss += torch.mean(torch.abs(v0[v0>0] - vk[v0>0]))
        s += 1.
    print(f'Epoch: {epoch} | Loss: {train_loss/s}') # Normalized Train Loss Per Epoch

Epoch: 1 | Loss: 0.28376778960227966
Epoch: 2 | Loss: 0.15265803039073944
Epoch: 3 | Loss: 0.15064960718154907
Epoch: 4 | Loss: 0.1497931182384491
Epoch: 5 | Loss: 0.13532854616641998
Epoch: 6 | Loss: 0.14557436108589172
Epoch: 7 | Loss: 0.1499110609292984
Epoch: 8 | Loss: 0.14636628329753876
Epoch: 9 | Loss: 0.14722248911857605
Epoch: 10 | Loss: 0.14895452558994293


#### Testing the Restricted Boltzmann Machine (RBM)

In [155]:
test_loss = 0
s = 0.
for user_id in range(num_users):
    v = training_set[user_id:user_id + 1]
    vt = test_set[user_id:user_id + 1]
    if len(vt[vt>0]) > 0:       
        _, h = rbm.sample_hidden(v)
        _, v = rbm.sample_visible(h)
        test_loss += torch.mean(torch.abs(vt[vt>0] - v[vt>0]))
        s += 1.
print(f'Test Loss: {test_loss/s}') # Normalized Test Loss

Test Loss: 0.14737629890441895
